In [3]:
from glob import glob
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
ls

 Volume in drive C is OS
 Volume Serial Number is EC28-98FA

 Directory of c:\Users\danus\Downloads\epilepsy_dl

23-09-2023  21:44    <DIR>          .
23-09-2023  21:49    <DIR>          ..
23-09-2023  22:15             2,846 eeg_ml.ipynb
23-09-2023  21:40    <DIR>          epilepsy_data
               1 File(s)          2,846 bytes
               3 Dir(s)  73,849,692,160 bytes free


In [5]:
all_files_path=glob('epilepsy_data/*.edf')
print(len(all_files_path))

28


In [6]:
all_files_path[0]

'epilepsy_data\\h01.edf'

In [17]:
healthy_file_path=[i for i in all_files_path if  'h' in i.split('\\')[1]]
patient_file_path=[i for i in all_files_path if  's' in i.split('\\')[1]]

In [18]:
healthy_file_path

['epilepsy_data\\h01.edf',
 'epilepsy_data\\h02.edf',
 'epilepsy_data\\h03.edf',
 'epilepsy_data\\h04.edf',
 'epilepsy_data\\h05.edf',
 'epilepsy_data\\h06.edf',
 'epilepsy_data\\h07.edf',
 'epilepsy_data\\h08.edf',
 'epilepsy_data\\h09.edf',
 'epilepsy_data\\h10.edf',
 'epilepsy_data\\h11.edf',
 'epilepsy_data\\h12.edf',
 'epilepsy_data\\h13.edf',
 'epilepsy_data\\h14.edf']

In [19]:
patient_file_path

['epilepsy_data\\s01.edf',
 'epilepsy_data\\s02.edf',
 'epilepsy_data\\s03.edf',
 'epilepsy_data\\s04.edf',
 'epilepsy_data\\s05.edf',
 'epilepsy_data\\s06.edf',
 'epilepsy_data\\s07.edf',
 'epilepsy_data\\s08.edf',
 'epilepsy_data\\s09.edf',
 'epilepsy_data\\s10.edf',
 'epilepsy_data\\s11.edf',
 'epilepsy_data\\s12.edf',
 'epilepsy_data\\s13.edf',
 'epilepsy_data\\s14.edf']

In [20]:
def read_data(file_path):
    datax = mne.io.read_raw_edf(file_path,preload = True)
    datax.set_eeg_reference()
    datax.filter(l_freq=1,h_freq=45) #bandpass filter
    epochs = mne.make_fixed_length_epochs(datax, duration=25,overlap = 0)
    epochs = epochs.get_data()
    return epochs    

In [21]:
data=read_data(healthy_file_path[0])

Extracting EDF parameters from c:\Users\danus\Downloads\epilepsy_dl\epilepsy_data\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
37 matching events found
No baseline correction applied
0 projection items

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [22]:
data.shape

(37, 19, 6250)

In [23]:
%%capture
control_epochs_array = [read_data(i) for i in healthy_file_path]
patient_file_array = [read_data(i) for i in patient_file_path]

In [28]:
control_epochs_array[1].shape

(36, 19, 6250)

In [25]:
data_list = control_epochs_array + patient_file_array

In [47]:
data_list[0].shape

(37, 19, 6250)

In [29]:
control_epochs_label = [len(i)*[0] for i in control_epochs_array]
patient_epochs_label = [len(i)*[1] for i in patient_file_array]

In [30]:
len(control_epochs_label),len(patient_epochs_label)

(14, 14)

In [31]:
label_list = control_epochs_label + patient_epochs_label

In [41]:
len(label_list),len(data_list)

(28, 28)

In [48]:
group_list = [[i]*len(j) for i,j in enumerate(data_list)]


In [49]:
len(group_list)

28

In [50]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(group_list)
print(data_array.shape,label_array.shape,group_array.shape)

(1142, 19, 6250) (1142,) (1142,)


In [51]:
data_array.shape,label_array.shape,group_array.shape

((1142, 19, 6250), (1142,), (1142,))

In [52]:
from scipy import stats

In [53]:
def mean(data):
    return np.mean(data,axis=-1)
    
def std(data):
    return np.std(data,axis=-1)

def ptp(data):
    return np.ptp(data,axis=-1)

def var(data):
        return np.var(data,axis=-1)

def minim(data):
      return np.min(data,axis=-1)


def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)
def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2,axis=-1))  

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)


def skewness(data):
    return stats.skew(data,axis=-1)

def kurtosis(data):
    return stats.kurtosis(data,axis=-1)

In [54]:
def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data)),axis=-1)

In [55]:
from tqdm import tqdm_notebook

In [56]:
features = []
for d in data_array:
    features.append(concatenate_features(d))

In [57]:
features=np.array(features)
features.shape

(1142, 247)

In [58]:
mean(d).shape

(19,)

In [60]:
features[0][1]

-5.056953150744914e-08

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV

In [64]:
clf=LogisticRegression()
gkf=GroupKFold(n_splits=5)
param_grid = {'classifier__C': [0.01,0.05,0.1,0.5, 1,2,3,4,5,8, 10,12,15]}
pipe=Pipeline([('scaler',StandardScaler()),('classifier',clf)])
gscv=GridSearchCV(pipe,param_grid,cv=gkf,n_jobs=16)
gscv.fit(features,label_array,groups=group_array)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', LogisticRegression())]),
             n_jobs=16,
             param_grid={'classifier__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5,
                                           8, 10, 12, 15]})

In [65]:
gscv.best_params_

{'classifier__C': 0.1}

In [66]:
gscv.best_score_

0.6675119125460665